# 1. Import packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [1]:
import os

In [2]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

23536


# 2. Load refgen df

In [3]:
codebook_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis'
df_refgen_savename = os.path.join(codebook_folder,'refgen_df_for_jie.csv')

In [4]:
df_refgen=pd.read_csv(df_refgen_savename,index_col=0)
df_refgen['chr'] = df_refgen['chr'].apply(lambda x: 'chr'+x)
df_refgen

,name,id,chr,chr_order,library,dtype,loci_name,start,end,hyb,delta,region_id
0,1:3742742-3759944,1,chr1,0,CTP11,combo,chr1_3742742_3759944,3742742,3759944,0,0,1
1,1:6245958-6258969,2,chr1,1,CTP11,combo,chr1_6245958_6258969,6245958,6258969,1,2503216,2
2,1:8740008-8759916,3,chr1,2,CTP11,combo,chr1_8740008_8759916,8740008,8759916,2,4997266,3
3,1:9627926-9637875,1,chr1,3,CTP13,combo,chr1_9627926_9637875,9627926,9637875,3,5885184,4
4,1:9799472-9811359,2,chr1,4,CTP13,combo,chr1_9799472_9811359,9799472,9811359,4,6056730,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1977,X:166247682-166259932,1059,chrX,60,CTP11,combo,chrX_166247682_166259932,166247682,166259932,60,159996393,1978
1978,X:167157164-167167452,990,chrX,61,CTP13,combo,chrX_167157164_167167452,167157164,167167452,61,160905875,1979
1979,X:168746045-168757590,1060,chrX,62,CTP11,combo,chrX_168746045_168757590,168746045,168757590,62,162494756,1980
1980,X:169963295-170005197,991,chrX,63,CTP13,combo,chrX_169963295_170005197,169963295,170005197,63,163712006,1981


# 3. Prepare candidate spots

In [5]:
decode_main_folder = r'\\10.245.74.158\Chromatin_NAS_8\Exported_data'
decode_exp_folder = os.path.join(decode_main_folder,'20220402-Export_new_v2')

## Process all fov

### shared params

In [6]:
pixel_z, pixel_x, pixel_y = 250, 108, 108

In [7]:
from tqdm import tqdm

In [8]:
# map info from df_ref_gen to coords; hyb should be the same as chromosome order
geneID_to_hyb = {name:hyb for name, hyb in zip(df_refgen['name'], df_refgen['hyb'])}    

def map_to_hyb(gene_name):
    return geneID_to_hyb[gene_name]

In [9]:
4240.806192/pixel_x
123492.8160/pixel_y
5762.010083/pixel_z
2832.196083/pixel_z

11.328784332

In [10]:
decoded_file_list = [os.path.join(decode_exp_folder, _fl) for _fl in os.listdir(decode_exp_folder) if '.hdf5' in _fl]
decoded_file_list[:3]

['\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\20220402-Export_new_v2\\Decoded_Fov-0.hdf5',
 '\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\20220402-Export_new_v2\\Decoded_Fov-1.hdf5',
 '\\\\10.245.74.158\\Chromatin_NAS_8\\Exported_data\\20220402-Export_new_v2\\Decoded_Fov-10.hdf5']

### start process

In [11]:
df_list = []
# ini file counter
file_counts = np.full((len(decoded_file_list),), False)
original_length_list = []
final_length_list = []
cell_ids_list = []

In [12]:
replicate_exp = '20220402'
for i, _file in tqdm(enumerate(decoded_file_list[:])):
    #check whether the file has been analyzed
    if file_counts[i]:
        continue
    file_counts[i] = True
    
    with h5py.File(_file, "r") as f:
        #print("Keys: %s" % f.keys())
        # get all cell keys to access the h5df
        cell_key_list = list(f.keys())
    
    # append coords of each cell from the same decoded fov
    _coords_df_fov_list=[]
    for _cell_key in cell_key_list:
        _coords_df = pd.read_hdf(_file, _cell_key)                   
        if not _coords_df.empty:
            _coords_df['orig_cellID']=_cell_key
            _coords_df['FOV']=_file.split('Fov-')[-1].split('.hdf5')[0]
            _coords_df['replicate']=replicate_exp
            _coords_df['chr'] = _coords_df['chr'].apply(lambda x: 'chr'+x)
            _coords_df_fov_list.append(_coords_df)
    
    _coords_df_fov = pd.concat(_coords_df_fov_list)

    _coords_df_fov.reset_index(inplace=True)
    _coords_df_fov.rename(columns={'center_z':'z_hat', 'center_y':'y_hat', 'center_x':'x_hat', 
                                    }, inplace = True)
    
    # spot num each fov
    original_length = len(_coords_df_fov)
    original_length_list.append(original_length)
        
    # convert physical locations to pixle...
    # since 1 z pixel is not equvilant to 1 x/y pixel, convert z pixel to pseudo pixel
    #_coords_df_fov.loc[:,'z_hat'] = (np.array(_coords_df_fov.loc[:,'z_hat'].values)/pixel_z)
    _coords_df_fov.loc[:,'z_hat'] = (np.array(_coords_df_fov.loc[:,'z_hat'].values)/pixel_z)*pixel_z/pixel_x
    _coords_df_fov.loc[:,'y_hat'] = np.array(_coords_df_fov.loc[:,'y_hat'].values)/pixel_y
    _coords_df_fov.loc[:,'x_hat'] = np.array(_coords_df_fov.loc[:,'x_hat'].values)/pixel_x

    #sigma = np.full((len(_coords_df_fov,),), 1)
    sigma = 1
    _coords_df_fov['sig_x'] = sigma
    _coords_df_fov['sig_y'] = sigma
    _coords_df_fov['sig_z'] = sigma

    # append info below if necessary (some are redundant..)
    # record hyb
    _coords_df_fov['hyb'] = _coords_df_fov['region_name'].apply(lambda x: map_to_hyb(x))
    # get other attributes for the dataframe
    _coords_df_fov['celltype'] = _coords_df_fov['subclass']
    # drop redundant info
    _coords_df_fov.drop(columns=['level_0','subclass'],inplace=True)

    
    # append each fov
    df_list.append(_coords_df_fov)
    final_length = len(_coords_df_fov)
    final_length_list.append(final_length)

    print(f'finishing processing {_file}')
    print(f'{final_length} is kept out of {original_length}')

1it [00:01,  1.09s/it]

finishing processing \\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2\Decoded_Fov-0.hdf5
31938 is kept out of 31938



2it [00:04,  2.73s/it]

finishing processing \\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2\Decoded_Fov-1.hdf5
382421 is kept out of 382421



3it [00:11,  4.38s/it]

...

finishing processing \\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2\Decoded_Fov-97.hdf5
360644 is kept out of 360644



158it [09:57,  3.71s/it]

finishing processing \\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2\Decoded_Fov-98.hdf5
256070 is kept out of 256070


159it [10:00,  3.78s/it]

finishing processing \\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2\Decoded_Fov-99.hdf5
276354 is kept out of 276354


In [13]:
# concat all decoded spots
#coord_df_exp = pd.concat(df_list)
#print(len(coord_df_exp))
print(len(final_length_list))
print(len(original_length_list))

159
159


### save df into subset chunks

In [17]:
# slow part; only save these when testing the script; would be more practical to process each fov hdf5-generated df direcly

In [14]:
len(df_list)

159

In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
chunk_ids_list = list(chunks(range(len(df_list)), 12))
chunk_ids_list

[range(0, 12),
 range(12, 24),
 range(24, 36),
 range(36, 48),
 range(48, 60),
 range(60, 72),
 range(72, 84),
 range(84, 96),
 range(96, 108),
 range(108, 120),
 range(120, 132),
 range(132, 144),
 range(144, 156),
 range(156, 159)]

In [20]:
#replicate_exp='20220402'
merged_spot_folder=os.path.join(r'\\10.245.74.158\Chromatin_NAS_8\Exported_data\jie_CW_decode', f'{replicate_exp}exp')

for _ct, _ids in tqdm(enumerate(chunk_ids_list)):
    coord_df_exp_subfname = os.path.join(merged_spot_folder, f'{replicate_exp}exp_cand_spots_part{_ct}.csv')
    sub_df_list = df_list[_ids[0]:_ids[-1]]
    #print(_ids)
    sub_dfs = pd.concat(sub_df_list)
    sub_dfs.to_csv(coord_df_exp_subfname, index=False)


14it [16:27, 70.56s/it]


In [21]:
del df_list